In [1]:
import torch
from torch import nn
from torch import optim
from torch import autograd as grad
import torch
import torchvision
from torchvision import transforms, datasets
import time
import random

This code classifies 10 different classes of images

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convL1 = nn.Conv2d(in_channels=3,out_channels=64, 
                                kernel_size=(3,3), stride= (1,1),padding=2)
        nn.init.kaiming_normal(self.convL1.weight)
        self.poolL1 = nn.MaxPool2d((2,2), (2,2))
        self.convL2 = nn.Conv2d(in_channels=64,out_channels=32, 
                                kernel_size=(2,2), stride= (2,1), padding=2)
        nn.init.kaiming_normal(self.convL2.weight)
        self.poolL2 = nn.MaxPool2d((2,2), (2,2))
        self.convL3 = nn.Conv2d(in_channels=32,out_channels=16, 
                                kernel_size=(1,1), stride= (1,1))
        nn.init.kaiming_normal(self.convL1.weight)
        self.poolL2 = nn.MaxPool2d((2,2), (2,2))
        self.convL3 = nn.Conv2d(in_channels=32,out_channels=16, 
                                kernel_size=(1,1), stride= (1,1))
        
        self.linL4 = nn.Linear(800, 10)
        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()
        self.Softmax = nn.Softmax()

    def forward(self, x):
        x=self.softplus(self.convL1(x))
        x=self.poolL1(x)
        x=self.softplus(self.convL2(x))
        x=self.poolL2(x)
        x=self.relu(self.convL3(x))

        x=torch.flatten(x)
        x=self.Softmax(self.linL4(x))

        return torch.tensor(x,dtype=torch.float32)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

Additional Functions

In [3]:
def ret_shape(training_set)->str:
    ret_string = ""
    for idx, data in enumerate(training_set):
            datas = data[0]
            labels = data[1]
            ret_string+="{}\n".format(datas.shape)
            ret_string+="Labels:{}\n".format(labels)
            ret_string+="Labels shape:{}\n".format(len(labels))
            ret_string+="Labels[0] shape:{}\n".format(labels[0].shape)
            break
    return ret_string


def max_index(max_tnsr):
    max_tnsr = max_tnsr.tolist()
    max_num = 0
    for num in max_tnsr:
        if max_num < num:
            max_num = num
    return max_tnsr.index(max_num)


Train Cell:

In [4]:
FILE_PATH = "model.pth"

def train_net(cnn, train_ds, dev):
    start_time = time.time
    iter = 0

    # Training loop part
    for sample in train_ds:

        targetY = torch.nn.functional.one_hot(
            torch.tensor(sample[1]), num_classes=10)
        targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)

        optimizer = optim.SGD(params=cnn.parameters(), lr=1e-3,
                          momentum=0.99, weight_decay=0.01)
        #optimizer = optim.Adam(params=cnn.parameters(), weight_decay=0.1
        #                       , lr=1e-3, betas=[0.9, 0.999])
        
        pred = cnn.forward(sample[0])
        loss_func = nn.CrossEntropyLoss()
        # ---BackPropagation---
        loss = loss_func(pred, targetY)
        loss.backward()
        optimizer.step()

        if iter%100==0:
            torch.save(cnn.cpu(), FILE_PATH)
            print(iter, " prediction: ",max_index(pred) == max_index(targetY))  

        optimizer.zero_grad()
        iter+=1


In [5]:
if __name__ == '__main__':

    # Accessing processing unit
    if torch.cuda.is_available() : device = "cuda:0"
    else : device = "cpu"
    device = torch.device(device)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
    net=CNN().to(device)

    # Loading the dataset
    train_data = torchvision.datasets.CIFAR10(
        root= "C:/Users/orian/OneDrive/שולחן העבודה/My Coding Files"
        ,train=True, transform=transforms.Compose([transforms.ToTensor()])
        #, target_transform=transforms.Compose([transforms.ToTensor()])
    )
    
    #train_data = DataLoader(train_data, shuffle=True)
    train_net(net, train_data, dev=device)


C:\Users\orian\AppData\Local\Temp\ipykernel_27636\2246012175.py:6: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  nn.init.kaiming_normal(self.convL1.weight)
C:\Users\orian\AppData\Local\Temp\ipykernel_27636\2246012175.py:10: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  nn.init.kaiming_normal(self.convL2.weight)
C:\Users\orian\AppData\Local\Temp\ipykernel_27636\2246012175.py:14: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  nn.init.kaiming_normal(self.convL1.weight)
C:\Users\orian\AppData\Local\Temp\ipykernel_27636\2684218324.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)
C:\Users\orian\AppData\Roaming\P

0  prediction:  False
100  prediction:  False
200  prediction:  False
300  prediction:  True
400  prediction:  True
500  prediction:  False
600  prediction:  False
700  prediction:  False
800  prediction:  True
900  prediction:  False
1000  prediction:  False
1100  prediction:  False
1200  prediction:  False
1300  prediction:  False
1400  prediction:  False
1500  prediction:  True
1600  prediction:  False
1700  prediction:  False
1800  prediction:  True
1900  prediction:  False
2000  prediction:  False
2100  prediction:  False
2200  prediction:  False
2300  prediction:  False
2400  prediction:  False
2500  prediction:  False
2600  prediction:  False
2700  prediction:  False
2800  prediction:  False
2900  prediction:  False
3000  prediction:  False
3100  prediction:  False
3200  prediction:  False
3300  prediction:  False
3400  prediction:  False
3500  prediction:  False
3600  prediction:  False
3700  prediction:  False
3800  prediction:  False
3900  prediction:  False
4000  prediction:

Test Cell:

In [6]:
FILE = "model.pth"
model = torch.load(FILE)
model.eval()

test_data = torchvision.datasets.CIFAR10(
        root= "C:/Users/orian/OneDrive/שולחן העבודה/My Coding Files"
        ,train=False, transform=transforms.Compose([transforms.ToTensor()]), 
    )
image_lst = []
i = random.randint(0,10000)
print(model.forward(test_data[i][0]))
print(max_index(model.forward(test_data[i][0]))," / ", test_data[i][1])

C:\Users\orian\AppData\Local\Temp\ipykernel_27636\594253079.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(FILE)


tensor([0.0905, 0.0567, 0.1871, 0.0856, 0.0760, 0.0919, 0.0603, 0.1001, 0.0863,
        0.1655])
2  /  9


C:\Users\orian\AppData\Local\Temp\ipykernel_27636\2246012175.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x,dtype=torch.float32)
